Tutorial: https://www.analyticsvidhya.com/blog/2019/07/pytorch-transformers-nlp-python/

# Buld sequence completion model using GPT-2

The code is straightforward: we tokenize and index the text as a sequence of numbers and pass it into the GPT2LMHeardModel. 

In [4]:
# ! pip install pytorch_transformers

In [5]:
import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

In [55]:
# load pre-trained model tokenizzer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Enconde a text inputs
text = "Do you want to"
indexed_tokens = tokenizer.encode(text)
print(len(index_tokens))
indexed_tokens

7


[2141, 345, 765, 284]

In [48]:
# convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([index_tokens])
tokens_tensor

tensor([[ 1867,   318,   262, 14162,  1097,   287,   262]])

In [49]:
# load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [50]:
# Set the model in evaluation mode to deactivate the DropOut modules
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [51]:
# if you have a GPU, put everything on cuda
# tokens_tensor = tokens_tensor.to('cuda')
# model.to('cuda')

In [52]:
# predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

In [53]:
# get the predicted next sub-word
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

In [54]:
print(predicted_text)

 Do you want to world


# Natural Language Generation using GPT-2, Transformer-XL, and XLNet
Generate a paragraph of text based on the given input.

# BERT
Predict masked words

In [56]:
import torch
from pytorch_transformers import BertTokenizer, BertModel, BertForMaskedLM

In [57]:
# load the pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 1352320.68B/s]


In [58]:
# tokenize input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

In [63]:
# maskt a token that will try to predict back with 'BertForMaskedML'
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

In [67]:
# convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

In [68]:
# convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([index_tokens])
segments_tensors = torch.tensor([segments_ids])

In [75]:
# load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

In [76]:
# Set the model in evaluation mode to deactivate the DropOut modules
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [78]:
# predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    predictions = outputs[0]

RuntimeError: The size of tensor a (7) must match the size of tensor b (14) at non-singleton dimension 1

In [79]:
# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'
print('Predicted token is:',predicted_token)


IndexError: index 8 is out of bounds for dimension 1 with size 7